In [1]:
import tensorflow as tf
import keras
import numpy as np

from keras import backend as K
from keras.layers.normalization import BatchNormalization
from keras.models import load_model

tf.__version__

'2.4.1'

## GPU and CPU settings
If GPU is not available, comment out the bottom block.

In [2]:
# If GPU is not available: 
# GPU_USE = '/cpu:0'
# config = tf.compat.v1.ConfigProto(device_count = {"GPU": 0})


# If GPU is available: 
config = tf.compat.v1.ConfigProto()
config.log_device_placement = True
config.allow_soft_placement = True
config.gpu_options.allocator_type = 'BFC'

# Limit the maximum memory used
config.gpu_options.per_process_gpu_memory_fraction = 0.1

# set session config
tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

Device mapping: no known devices.


## Define input and output

In [23]:
modelname = "myModelCNN"

input_path = "./"
input_file = modelname + ".h5"

output_path = "/var/www/html/tutorial-data/"
output_file = modelname + ".pb"

output_node_prefix = "output_node"

In [24]:
K.set_learning_phase(0)
net_model = load_model(input_path + input_file)

num_output = 1
pred = [None]*num_output
pred_node_names = [None]*num_output
for i in range(num_output):
    pred_node_names[i] = output_node_prefix+str(i)
    pred[i] = tf.identity(net_model.outputs[i], name=pred_node_names[i])
print('output nodes names are: ', pred_node_names)
output_node_prefix = pred_node_names[0]

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/backend.py:434: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


output nodes names are:  ['output_node0']


In [20]:
sess = tf.compat.v1.keras.backend.get_session()

## Export model

In [25]:
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io
tf.compat.v1.disable_eager_execution()
constant_graph = tf.compat.v1.graph_util.convert_variables_to_constants(sess, sess.graph.as_graph_def(), pred_node_names)

graph_io.write_graph(constant_graph, output_path, output_file, as_text=False)

print('Saved the freezed graph at: ', (output_path + output_file))

Saved the freezed graph at:  /var/www/html/tutorial-data/myModelCNN.pb


## Show input and output node

In [26]:
g = tf.compat.v1.GraphDef()
g.ParseFromString(open(output_path + output_file, "rb").read())
s = ""
for n in g.node:
    s =s + str(n.name) + "\n"

print(s)

dense_input
dense_1/kernel
dense_1/bias
dense_1/MatMul/ReadVariableOp
dense_1/MatMul
dense_1/BiasAdd/ReadVariableOp
dense_1/BiasAdd
dense_1/Relu
dense_1_1/kernel
dense_1_1/bias
dense_1_1/MatMul/ReadVariableOp
dense_1_1/MatMul
dense_1_1/BiasAdd/ReadVariableOp
dense_1_1/BiasAdd
dense_1_1/Softmax
output_node0

